In [ ]:
import os, sys, glob, time

import numpy as np
import pandas as pd

import math

# Parallel proces
from myparproc.ParProc import ParProc
from myparproc.ProgressCheck import ProgressCheck

# Linear algebra
import myutil.myutil as util

# Plotting tools
import matplotlib.pyplot as plt

import rospy, rosbag

# For logging to csv
import csv

from livox_ros_driver.msg import CustomMsg as lvCloud
from livox_ros_driver.msg import CustomPoint as lvPoint
from livox_ros_driver2.msg import CustomMsg as lv2Cloud
from livox_ros_driver2.msg import CustomPoint as lv2Point

# For kd tree search
# import pcl, pcl_ros, pcl_msgs

# For load/unloading pcd/ply
import pypcd
from pypcd import pypcd

# For loading/unloading pcd/ply
# from pyntcloud import PyntCloud

# Some processing or plotting
import open3d as o3d

from ceva import Ceva

# Lidar topic
lidar_topic = ['/livox/lidar_192_168_10_110', '/livox/lidar_192_168_11_125']

# Location of the dataset
bag_file = '/media/tmn/mySataSSD1/Experiments/gptr/cathhs/cathhs_07.bag'

# Output location
output_dir = '/media/tmn/mySataSSD1/Experiments/gptr/' + os.path.basename(bag_file).replace('.bag', '')
print(output_dir)

# Fields to export to ply
interested_fields = ['x', 'y', 'z', 'intensity', 't']

# Transform from the vicon center to the sensor
T_V_S = [np.array([[ 1,  0,  0,  0    ],
                   [ 0,  1,  0,  0    ],
                   [ 0,  0,  1, -0.025 ],
                   [ 0,  0,  0,  1    ]]),
         np.array([[ 0,  0,  1,  0.127],
                   [ 0,  1,  0, -0.014],
                   [-1,  0,  0, -0.146],
                   [ 0,  0,  0,  0    ]]),
        ]


In [ ]:
output_dir_topic = {}

import subprocess
for lidx, topic in enumerate(lidar_topic):
    output_dir_topic[topic] = output_dir + f'/clouds/__livox__lidar_{lidx}__points'
    print(output_dir_topic[topic])
    os.makedirs(output_dir_topic[topic], exist_ok=True)
    # !rosrun pcl_ros bag_to_pcd {bag_file} {topic} {output_dir_}
    # process = subprocess.run(["rosrun", "pcl_ros", "bag_to_pcd", bag_file, topic, output_dir_topic], check=True, capture_output=False)

In [8]:
# import subprocess
# for topic in lidar_topic:
#     output_dir_topic = output_dir + '/clouds/' + topic.replace('/', '__')
#     clouds = glob.glob(output_dir_topic + '/**/*.pcd', recursive=True)
#     for cloud in clouds:
#         pc = pypcd.PointCloud.from_path(cloud).pc_data
#         t = pc['timestamp'].astype(int)
#         t = t -  t[0]
#         pcnew = np.array(list(zip(pc['x'], pc['y'], pc['z'], pc['intensity'], t)), dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('intensity', '<f4'), ('t', 'u4')])
#         pypcd.PointCloud.from_array(pcnew).save_pcd(cloud)



In [ ]:
def lvCloudToArray(msg, cloud_idx):
    x = []
    y = []
    z = []
    i = []
    t = []
    
    for lvP in msg.points:
        x.append(lvP.x)
        y.append(lvP.y)
        z.append(lvP.z)
        i.append(lvP.reflectivity)
        t.append(lvP.offset_time)

    points = list(zip(x, y, z, i, t))
    points = np.array(points, dtype=[('x',         'f4'),
                                     ('y',         'f4'),
                                     ('z',         'f4'),
                                     ('intensity', 'f4'),
                                     ('t',         'u4'),
                                    ])

    return points

# Load the pointcloud in the bag
bag = rosbag.Bag(bag_file)

# Get total message count
total_msg = bag.get_message_count(lidar_topic)
count_max_digit = len(str(total_msg))
print("Total msg: ", total_msg, count_max_digit)

progress = 0
count = -1
for topic, msg, t in bag.read_messages():
    if topic in lidar_topic:
    
        # Increment counter
        count += 1

        # Timestamp
        timestamp = msg.header.stamp.to_sec()

        # if topic == lidar_topic[0]:
        #     print(timestamp)

        # Extract the point cloud from message
        lvpc_in = lvCloudToArray(msg, count)
        
        # Sort points by sampling time, then ring (lexsort uses the reverse order)
        lvpc_in = lvpc_in[np.argsort(lvpc_in, order='t')]
       
        # Make the file name
        lvpc_filename = output_dir_topic[topic] + '/' + f'{timestamp:.9f}' + '.pcd'
        
        # Save the pointcloud
        pypcd.PointCloud.from_array(lvpc_in).save_pcd(lvpc_filename, compression='binary')
        
        # Report the progress
        if (math.floor(count/total_msg*100) != progress or count == total_msg - 1):
            progress = math.floor(count/total_msg*100)
            print(f'Export {progress:3d}%')
        # print(f'Seq: {seq}. Cloud {count}. SweepTime: {sweepTime}')

# Need to close the log file to get the text out
bag.close()


In [10]:
gtr_topic = '/vicon/cathhs/cathhs'
gtr_txt = [[] for topic in lidar_topic]
gtr_pcd = [[] for topic in lidar_topic]

os.makedirs(output_dir + '/gtr/', exist_ok=True)

# Load the pointcloud in the bag
bag = rosbag.Bag(bag_file)

# # Get total message count
# total_msg = bag.get_message_count(lidar_topic)
# count_max_digit = len(str(total_msg))
# print("Total msg: ", total_msg, count_max_digit)

progress = 0
count = -1
for topic, msg, t in bag.read_messages():
    if topic == gtr_topic:
    
        # Increment counter
        count += 1

        # Timestamp
        timestamp = msg.header.stamp.to_sec()

        p = msg.transform.translation
        q = msg.transform.rotation

        for lidx, topic in enumerate(lidar_topic):
            P_W_V = np.array([p.x, p.y, p.z]).reshape((3, 1))
            Q_W_V = np.array([q.w, q.x, q.y, q.z]).reshape((4))

            P_W_S = list(P_W_V + (util.quat2rotm(Q_W_V)@T_V_S[lidx][0:3, 3]).reshape((3, 1)))
            Q_W_S = list(util.rotm2quat(util.quat2rotm(Q_W_V)@T_V_S[lidx][0:3, 0:3]))

            # Extract the point cloud from message
            gtr_txt[lidx].append((count, timestamp, P_W_S[0], P_W_S[1], P_W_S[2], Q_W_S[1], Q_W_S[2], Q_W_S[3], Q_W_S[0]))
            gtr_pcd[lidx].append((P_W_S[0], P_W_S[1], P_W_S[2], count, timestamp, Q_W_S[1], Q_W_S[2], Q_W_S[3], Q_W_S[0]))


# Need to close the log file to get the text out
bag.close()

for lidx, gt in enumerate(gtr_txt):
    if len(gt) > 0:
        gt = np.array(gt, dtype=[('intensity', '<f4'), ('t', '<f8'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('qx', '<f4'), ('qy', '<f4'), ('qz', '<f4'), ('qw', '<f4')])
        np.savetxt(output_dir + f'/gtr/lidar_{lidx}_gtr.txt', gt, delimiter=' ')

for lidx, gt in enumerate(gtr_pcd):
    if len(gt) > 0:
        gt = np.array(gt, dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('intensity', '<f4'), ('t', '<f8'), ('qx', '<f4'), ('qy', '<f4'), ('qz', '<f4'), ('qw', '<f4')])
        pypcd.PointCloud.from_array(gt).save_pcd(output_dir + f'/gtr/lidar_{lidx}_gtr.pcd')